In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 5.4 MB/s eta 0:00:00


# optuna를 사용하여 AutoML 파이프라인을 코드로 구현

# 구현(끼워맞추기)

In [37]:
from typing import Dict, Any, List, Union
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import optuna

In [3]:
def search_hyperparam(trial: optuna.trial.Trial) -> Dict[str, Any]:
    """Optuna를 사용하여 하이퍼파라미터를 탐색"""
    return {
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64])
    }

In [4]:
def search_model(trial: optuna.trial.Trial) -> nn.Module:
    """Optuna를 사용하여 모델 구조를 탐색"""
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    model = nn.Sequential(
        nn.Linear(10, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, 2)
    )
    return model

In [5]:
def train_model(
	model: nn.Module,
	hyperparams:Dict[str, Any]
	) -> nn.Module:
  pass

In [6]:
# search_hyperparam 과 search_model 함수를 사용하여 주어진 탐색 공간에서 모델과 하이퍼파라미터를 가져옴

In [7]:
def train_model(model: nn.Module, hyperparams: Dict[str, Any], train_loader: DataLoader, epochs: int = 10) -> nn.Module:
    """모델을 학습하는 함수"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, hyperparams["optimizer"])(model.parameters(), lr=hyperparams["lr"])
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in train_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            targets = targets.squeeze()  # (batch_size, 1) → (batch_size,)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")
    return model

In [8]:
# 주어진 모델과 하이퍼파라미터를 사용하여 모델을 학습

In [9]:
def evaluate(model: nn.Module) -> float:
    """학습된 모델을 평가하는 함수"""
    return torch.rand(1).item()

In [10]:
# 학습된 모델의 성능을 측정

In [13]:
def objective(trial: optuna.trial.Trial) -> float:
    """Optuna 최적화 목표 함수"""
    hyperparams = search_hyperparam(trial)
    model = search_model(trial)
    train_loader = DataLoader([(torch.rand(10), torch.randint(0, 2, (1,))) for _ in range(100)], batch_size=hyperparams["batch_size"], shuffle=True)
    trained_model = train_model(model, hyperparams, train_loader)
    score = evaluate(trained_model)
    return score

In [14]:
if __name__ == "__main__":
  study = optuna.create_study(direction="maximize")
  study.optimize(objective, n_trials=500)
  print(f"Best trial {study.best_trial}")

[I 2025-11-11 03:52:21,928] A new study created in memory with name: no-name-803562fb-f552-41ff-bee6-e09d80fb51e4
/tmp/ipython-input-4150312703.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-11-11 03:52:29,081] Trial 0 finished with value: 0.8743774890899658 and parameters: {'lr': 1.024984756467225e-05, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 170}. Best is trial 0 with value: 0.8743774890899658.
/tmp/ipython-input-4150312703.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-11-11 03:52:29,145] Trial 1 finished with value: 0.088289260864

Epoch 1, Loss: 0.7138
Epoch 2, Loss: 0.7013
Epoch 3, Loss: 0.6813
Epoch 4, Loss: 0.7111
Epoch 5, Loss: 0.6916
Epoch 6, Loss: 0.6888
Epoch 7, Loss: 0.6844
Epoch 8, Loss: 0.6989
Epoch 9, Loss: 0.6919
Epoch 10, Loss: 0.6991
Epoch 1, Loss: 0.6961
Epoch 2, Loss: 0.6903
Epoch 3, Loss: 0.6932
Epoch 4, Loss: 0.6928
Epoch 5, Loss: 0.6933
Epoch 6, Loss: 0.6926
Epoch 7, Loss: 0.6925
Epoch 8, Loss: 0.6913
Epoch 9, Loss: 0.6926
Epoch 10, Loss: 0.6914
Epoch 1, Loss: 0.6823
Epoch 2, Loss: 0.7038
Epoch 3, Loss: 0.7083
Epoch 4, Loss: 0.6776
Epoch 5, Loss: 0.6930
Epoch 6, Loss: 0.7145
Epoch 7, Loss: 0.6581
Epoch 8, Loss: 0.6642
Epoch 9, Loss: 0.6785
Epoch 10, Loss: 0.6790
Epoch 1, Loss: 0.6841
Epoch 2, Loss: 0.6798
Epoch 3, Loss: 0.6900
Epoch 4, Loss: 0.6902
Epoch 5, Loss: 0.6871
Epoch 6, Loss: 0.7010
Epoch 7, Loss: 0.6830
Epoch 8, Loss: 0.6895


[I 2025-11-11 03:52:29,265] Trial 3 finished with value: 0.2898746132850647 and parameters: {'lr': 6.488655772628353e-05, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 127}. Best is trial 0 with value: 0.8743774890899658.
[I 2025-11-11 03:52:29,301] Trial 4 finished with value: 0.6424998044967651 and parameters: {'lr': 0.0015265649536597753, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 118}. Best is trial 0 with value: 0.8743774890899658.
[I 2025-11-11 03:52:29,377] Trial 5 finished with value: 0.873277485370636 and parameters: {'lr': 1.3282573887630916e-05, 'optimizer': 'RMSprop', 'batch_size': 16, 'hidden_size': 246}. Best is trial 0 with value: 0.8743774890899658.
[I 2025-11-11 03:52:29,439] Trial 6 finished with value: 0.8427199125289917 and parameters: {'lr': 0.00019780144507721773, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 144}. Best is trial 0 with value: 0.8743774890899658.


Epoch 9, Loss: 0.6816
Epoch 10, Loss: 0.6917
Epoch 1, Loss: 0.7057
Epoch 2, Loss: 0.7163
Epoch 3, Loss: 0.6998
Epoch 4, Loss: 0.6913
Epoch 5, Loss: 0.7223
Epoch 6, Loss: 0.6873
Epoch 7, Loss: 0.7059
Epoch 8, Loss: 0.7048
Epoch 9, Loss: 0.6991
Epoch 10, Loss: 0.7041
Epoch 1, Loss: 0.7044
Epoch 2, Loss: 0.7000
Epoch 3, Loss: 0.7112
Epoch 4, Loss: 0.7078
Epoch 5, Loss: 0.7152
Epoch 6, Loss: 0.7075
Epoch 7, Loss: 0.7055
Epoch 8, Loss: 0.7041
Epoch 9, Loss: 0.7080
Epoch 10, Loss: 0.6986
Epoch 1, Loss: 0.7527
Epoch 2, Loss: 0.7655
Epoch 3, Loss: 0.7505
Epoch 4, Loss: 0.7379
Epoch 5, Loss: 0.7620
Epoch 6, Loss: 0.7198
Epoch 7, Loss: 0.7334
Epoch 8, Loss: 0.7776
Epoch 9, Loss: 0.7573
Epoch 10, Loss: 0.7569
Epoch 1, Loss: 0.7074
Epoch 2, Loss: 0.6974


[I 2025-11-11 03:52:29,532] Trial 7 finished with value: 0.6254012584686279 and parameters: {'lr': 0.000561113848051653, 'optimizer': 'RMSprop', 'batch_size': 16, 'hidden_size': 95}. Best is trial 0 with value: 0.8743774890899658.
[I 2025-11-11 03:52:29,613] Trial 8 finished with value: 0.5754837393760681 and parameters: {'lr': 5.6861877310725054e-05, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 182}. Best is trial 0 with value: 0.8743774890899658.
[I 2025-11-11 03:52:29,639] Trial 9 finished with value: 0.5562554597854614 and parameters: {'lr': 0.0109079073566523, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 255}. Best is trial 0 with value: 0.8743774890899658.


...

Epoch 1, Loss: 0.6884
Epoch 2, Loss: 0.6967
Epoch 3, Loss: 0.6954
Epoch 4, Loss: 0.6957
Epoch 5, Loss: 0.6882
Epoch 6, Loss: 0.6998
Epoch 7, Loss: 0.7028
Epoch 8, Loss: 0.6834
Epoch 9, Loss: 0.7061
Epoch 10, Loss: 0.6875
Epoch 1, Loss: 0.6881
Epoch 2, Loss: 0.7072
Epoch 3, Loss: 0.7042
Epoch 4, Loss: 0.6928
Epoch 5, Loss: 0.6865
Epoch 6, Loss: 0.6942
Epoch 7, Loss: 0.6948
Epoch 8, Loss: 0.7073
Epoch 9, Loss: 0.6964
Epoch 10, Loss: 0.7145
Epoch 1, Loss: 0.7243
Epoch 2, Loss: 0.7418
Epoch 3, Loss: 0.7295
Epoch 4, Loss: 0.7594
Epoch 5, Loss: 0.7234
Epoch 6, Loss: 0.7830
Epoch 7, Loss: 0.7273
Epoch 8, Loss: 0.7370
Epoch 9, Loss: 0.7012
Epoch 10, Loss: 0.6856
Epoch 1, Loss: 0.6920
Epoch 2, Loss: 0.7067
Epoch 3, Loss: 0.7148
Epoch 4, Loss: 0.7141
Epoch 5, Loss: 0.7025
Epoch 6, Loss: 0.7353


[I 2025-11-11 03:53:00,875] Trial 494 finished with value: 0.8041110634803772 and parameters: {'lr': 2.421425439911925e-05, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 157}. Best is trial 235 with value: 0.9996119141578674.
[I 2025-11-11 03:53:00,932] Trial 495 finished with value: 0.3978036046028137 and parameters: {'lr': 1.4097739902662423e-05, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 237}. Best is trial 235 with value: 0.9996119141578674.
[I 2025-11-11 03:53:01,001] Trial 496 finished with value: 0.23032814264297485 and parameters: {'lr': 0.00010188742737361532, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 256}. Best is trial 235 with value: 0.9996119141578674.


Epoch 7, Loss: 0.6933
Epoch 8, Loss: 0.6900
Epoch 9, Loss: 0.7336
Epoch 10, Loss: 0.7155
Epoch 1, Loss: 0.6917
Epoch 2, Loss: 0.6457
Epoch 3, Loss: 0.6459
Epoch 4, Loss: 0.7385
Epoch 5, Loss: 0.7347
Epoch 6, Loss: 0.6456
Epoch 7, Loss: 0.6460
Epoch 8, Loss: 0.7205
Epoch 9, Loss: 0.6714
Epoch 10, Loss: 0.6882
Epoch 1, Loss: 0.6929
Epoch 2, Loss: 0.6991
Epoch 3, Loss: 0.6955
Epoch 4, Loss: 0.7004
Epoch 5, Loss: 0.7001
Epoch 6, Loss: 0.6892
Epoch 7, Loss: 0.6935
Epoch 8, Loss: 0.7026
Epoch 9, Loss: 0.6943
Epoch 10, Loss: 0.6984
Epoch 1, Loss: 0.6828
Epoch 2, Loss: 0.7079
Epoch 3, Loss: 0.7187
Epoch 4, Loss: 0.7020
Epoch 5, Loss: 0.7166
Epoch 6, Loss: 0.6803
Epoch 7, Loss: 0.6997
Epoch 8, Loss: 0.6856
Epoch 9, Loss: 0.6878
Epoch 10, Loss: 0.7058


[I 2025-11-11 03:53:01,060] Trial 497 finished with value: 0.07943427562713623 and parameters: {'lr': 2.9155282256702038e-05, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 247}. Best is trial 235 with value: 0.9996119141578674.
[I 2025-11-11 03:53:01,133] Trial 498 finished with value: 0.8791454434394836 and parameters: {'lr': 0.0005469965744860481, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 184}. Best is trial 235 with value: 0.9996119141578674.
[I 2025-11-11 03:53:01,180] Trial 499 finished with value: 0.03941410779953003 and parameters: {'lr': 4.597917803467241e-05, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 226}. Best is trial 235 with value: 0.9996119141578674.


Epoch 1, Loss: 0.6908
Epoch 2, Loss: 0.6908
Epoch 3, Loss: 0.6885
Epoch 4, Loss: 0.6671
Epoch 5, Loss: 0.6842
Epoch 6, Loss: 0.6669
Epoch 7, Loss: 0.6698
Epoch 8, Loss: 0.6779
Epoch 9, Loss: 0.6612
Epoch 10, Loss: 0.6320
Epoch 1, Loss: 0.7071
Epoch 2, Loss: 0.7073
Epoch 3, Loss: 0.7121
Epoch 4, Loss: 0.7110
Epoch 5, Loss: 0.7100
Epoch 6, Loss: 0.7086
Epoch 7, Loss: 0.7084
Epoch 8, Loss: 0.7093
Epoch 9, Loss: 0.7070
Epoch 10, Loss: 0.7122
Best trial FrozenTrial(number=235, state=<TrialState.COMPLETE: 1>, values=[0.9996119141578674], datetime_start=datetime.datetime(2025, 11, 11, 3, 52, 43, 386401), datetime_complete=datetime.datetime(2025, 11, 11, 3, 52, 43, 436600), params={'lr': 2.588542356053925e-05, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 246}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'optimizer': CategoricalDistribution(choices=('Adam', 'SGD', 'RMSprop')), 'batch_size': Cat

In [15]:
# Optuna가 결과를 가지고 다음 모델과 하이퍼파라미터를 탐색

# 구현 실제

In [16]:
def objective(trial: optuna.trial.Trial) -> float:
    """Optuna 최적화 목표 함수"""
    hyperparams = search_hyperparam(trial)
    model = search_model(trial)
    train_loader = DataLoader([(torch.rand(10), torch.randint(0, 2, (1,))) for _ in range(100)], batch_size=hyperparams["batch_size"], shuffle=True)
    trained_model = train_model(model, hyperparams, train_loader)
    score = evaluate(trained_model)
    return score

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-11-11 03:53:27,355] A new study created in memory with name: no-name-da83e8ab-62e5-43e0-9349-e83b9a597b13
/tmp/ipython-input-4150312703.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-11-11 03:53:27,387] Trial 0 finished with value: 0.9761271476745605 and parameters: {'lr': 2.973287557381359e-05, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 213}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,444] Trial 1 finished with value: 0.7039942741394043 and parameters: {'lr': 0.025245665500884917, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 178}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,472] Trial 2 finished with value: 0.6721591353416443 and parameters: {'lr': 0.025286806599898237, 'optimizer': 'R

Epoch 1, Loss: 0.6985
Epoch 2, Loss: 0.6987
Epoch 3, Loss: 0.6934
Epoch 4, Loss: 0.6945
Epoch 5, Loss: 0.6970
Epoch 6, Loss: 0.6979
Epoch 7, Loss: 0.6975
Epoch 8, Loss: 0.6969
Epoch 9, Loss: 0.6962
Epoch 10, Loss: 0.6974
Epoch 1, Loss: 0.6897
Epoch 2, Loss: 0.6898
Epoch 3, Loss: 0.7094
Epoch 4, Loss: 0.6920
Epoch 5, Loss: 0.6927
Epoch 6, Loss: 0.6883
Epoch 7, Loss: 0.6973
Epoch 8, Loss: 0.6930
Epoch 9, Loss: 0.6935
Epoch 10, Loss: 0.6941
Epoch 1, Loss: 3.9786
Epoch 2, Loss: 3.0690
Epoch 3, Loss: 0.9136
Epoch 4, Loss: 0.7717
Epoch 5, Loss: 0.7093
Epoch 6, Loss: 0.7173
Epoch 7, Loss: 0.6876
Epoch 8, Loss: 0.7030
Epoch 9, Loss: 0.6887
Epoch 10, Loss: 0.6803
Epoch 1, Loss: 0.7200
Epoch 2, Loss: 0.7116
Epoch 3, Loss: 0.7127
Epoch 4, Loss: 0.7092
Epoch 5, Loss: 0.7030
Epoch 6, Loss: 0.7038
Epoch 7, Loss: 0.7014
Epoch 8, Loss: 0.7004
Epoch 9, Loss: 0.6965
Epoch 10, Loss: 0.6961
Epoch 1, Loss: 0.6961
Epoch 2, Loss: 0.6988
Epoch 3, Loss: 0.6963
Epoch 4, Loss: 0.6956
Epoch 5, Loss: 0.6948
Epoch 

[I 2025-11-11 03:53:27,576] Trial 4 finished with value: 0.9058969020843506 and parameters: {'lr': 2.0773683826382014e-05, 'optimizer': 'Adam', 'batch_size': 64, 'hidden_size': 87}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,671] Trial 5 finished with value: 0.08290719985961914 and parameters: {'lr': 0.04664953616721747, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 92}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,757] Trial 6 finished with value: 0.5924233198165894 and parameters: {'lr': 4.7094178892844704e-05, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 174}. Best is trial 0 with value: 0.9761271476745605.


Epoch 9, Loss: 0.6966
Epoch 10, Loss: 0.6999
Epoch 1, Loss: 0.7541
Epoch 2, Loss: 0.7008
Epoch 3, Loss: 0.7078
Epoch 4, Loss: 0.6720
Epoch 5, Loss: 0.6540
Epoch 6, Loss: 0.6341
Epoch 7, Loss: 0.6678
Epoch 8, Loss: 0.6362
Epoch 9, Loss: 0.6577
Epoch 10, Loss: 0.6459
Epoch 1, Loss: 0.7377
Epoch 2, Loss: 0.7094
Epoch 3, Loss: 0.7200
Epoch 4, Loss: 0.7114
Epoch 5, Loss: 0.6881
Epoch 6, Loss: 0.6841
Epoch 7, Loss: 0.7152
Epoch 8, Loss: 0.7099
Epoch 9, Loss: 0.6798
Epoch 10, Loss: 0.6871


[I 2025-11-11 03:53:27,877] Trial 7 finished with value: 0.22253602743148804 and parameters: {'lr': 0.01175486754836598, 'optimizer': 'RMSprop', 'batch_size': 16, 'hidden_size': 231}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,921] Trial 8 finished with value: 0.6958580017089844 and parameters: {'lr': 0.0001272469445568173, 'optimizer': 'RMSprop', 'batch_size': 64, 'hidden_size': 98}. Best is trial 0 with value: 0.9761271476745605.
[I 2025-11-11 03:53:27,950] Trial 9 finished with value: 0.30024945735931396 and parameters: {'lr': 0.004285804924396625, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 185}. Best is trial 0 with value: 0.9761271476745605.


Epoch 1, Loss: 2.6206
Epoch 2, Loss: 0.9429
Epoch 3, Loss: 0.8592
Epoch 4, Loss: 0.7204
Epoch 5, Loss: 0.6955
Epoch 6, Loss: 0.6847
Epoch 7, Loss: 0.6586
Epoch 8, Loss: 0.6672
Epoch 9, Loss: 0.6677
Epoch 10, Loss: 0.6407
Epoch 1, Loss: 0.6990
Epoch 2, Loss: 0.6977
Epoch 3, Loss: 0.6942
Epoch 4, Loss: 0.6937
Epoch 5, Loss: 0.6941
Epoch 6, Loss: 0.6897
Epoch 7, Loss: 0.6957
Epoch 8, Loss: 0.6897
Epoch 9, Loss: 0.6917
Epoch 10, Loss: 0.6911
Epoch 1, Loss: 0.6901
Epoch 2, Loss: 0.6876
Epoch 3, Loss: 0.6888
Epoch 4, Loss: 0.6860
Epoch 5, Loss: 0.6916
Epoch 6, Loss: 0.6912
Epoch 7, Loss: 0.6947
Epoch 8, Loss: 0.6879
Epoch 9, Loss: 0.6870
Epoch 10, Loss: 0.6870


In [18]:
# 학습된 모델의 성능을 측정한 이후
# Optuna가 측정된 성능을 기반으로 다음 모델과 하이퍼파라미터를 탐색

In [19]:
print(f"Best trial: {study.best_trial}")

Best trial: FrozenTrial(number=0, state=<TrialState.COMPLETE: 1>, values=[0.9761271476745605], datetime_start=datetime.datetime(2025, 11, 11, 3, 53, 27, 357062), datetime_complete=datetime.datetime(2025, 11, 11, 3, 53, 27, 387015), params={'lr': 2.973287557381359e-05, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 213}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'optimizer': CategoricalDistribution(choices=('Adam', 'SGD', 'RMSprop')), 'batch_size': CategoricalDistribution(choices=(16, 32, 64)), 'hidden_size': IntDistribution(high=256, log=False, low=32, step=1)}, trial_id=0, value=None)


In [20]:
if __name__ == "__main__":

# 1. Optuna Study 생성 (blackbox Optimizer 및 관리 담당)
  study = optuna.create_study(direction='maximize')

# 2. Study에 최적화할 목적함수 및 시도 횟수, 조건 등을 주고 Optimize
  study.optimize(objective, n_trials=500)
  print(f"Best trial {study.best_trial}")

[I 2025-11-11 03:53:40,557] A new study created in memory with name: no-name-7dcdc0da-3056-42f5-89c8-dbf7b658dc3e
/tmp/ipython-input-4150312703.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-11-11 03:53:40,656] Trial 0 finished with value: 0.3291028141975403 and parameters: {'lr': 0.0005816298076524294, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 98}. Best is trial 0 with value: 0.3291028141975403.
[I 2025-11-11 03:53:40,733] Trial 1 finished with value: 0.6410734057426453 and parameters: {'lr': 2.51724983084137e-05, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 126}. Best is trial 1 with value: 0.6410734057426453.


Epoch 1, Loss: 0.6991
Epoch 2, Loss: 0.6996
Epoch 3, Loss: 0.6994
Epoch 4, Loss: 0.7090
Epoch 5, Loss: 0.7161
Epoch 6, Loss: 0.6908
Epoch 7, Loss: 0.7094
Epoch 8, Loss: 0.7062
Epoch 9, Loss: 0.6984
Epoch 10, Loss: 0.6943
Epoch 1, Loss: 0.7187
Epoch 2, Loss: 0.7088
Epoch 3, Loss: 0.7125
Epoch 4, Loss: 0.7065
Epoch 5, Loss: 0.7177
Epoch 6, Loss: 0.7255
Epoch 7, Loss: 0.7149
Epoch 8, Loss: 0.7157
Epoch 9, Loss: 0.7261
Epoch 10, Loss: 0.7088
Epoch 1, Loss: 0.7448
Epoch 2, Loss: 0.7387
Epoch 3, Loss: 0.7463
Epoch 4, Loss: 0.7408
Epoch 5, Loss: 0.7452
Epoch 6, Loss: 0.7484
Epoch 7, Loss: 0.7517
Epoch 8, Loss: 0.7385


[I 2025-11-11 03:53:40,790] Trial 2 finished with value: 0.9351345896720886 and parameters: {'lr': 1.819216598595298e-05, 'optimizer': 'Adam', 'batch_size': 64, 'hidden_size': 137}. Best is trial 2 with value: 0.9351345896720886.
[I 2025-11-11 03:53:40,866] Trial 3 finished with value: 0.05923879146575928 and parameters: {'lr': 0.0003255625062325891, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 228}. Best is trial 2 with value: 0.9351345896720886.
[I 2025-11-11 03:53:40,948] Trial 4 finished with value: 0.5408924221992493 and parameters: {'lr': 0.0031395747419467905, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 89}. Best is trial 2 with value: 0.9351345896720886.


Epoch 9, Loss: 0.7466
Epoch 10, Loss: 0.7468
Epoch 1, Loss: 0.7082
Epoch 2, Loss: 0.6929
Epoch 3, Loss: 0.6837
Epoch 4, Loss: 0.7065
Epoch 5, Loss: 0.6900
Epoch 6, Loss: 0.6785
Epoch 7, Loss: 0.6923
Epoch 8, Loss: 0.6856
Epoch 9, Loss: 0.6744
Epoch 10, Loss: 0.6793
Epoch 1, Loss: 0.7041
Epoch 2, Loss: 0.6948
Epoch 3, Loss: 0.6983
Epoch 4, Loss: 0.6667
Epoch 5, Loss: 0.6791
Epoch 6, Loss: 0.6766
Epoch 7, Loss: 0.6974
Epoch 8, Loss: 0.6768
Epoch 9, Loss: 0.6828
Epoch 10, Loss: 0.6855
Epoch 1, Loss: 0.7393
Epoch 2, Loss: 0.6878
Epoch 3, Loss: 0.6827


[I 2025-11-11 03:53:41,037] Trial 5 finished with value: 0.748960554599762 and parameters: {'lr': 4.121514271653134e-05, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 245}. Best is trial 2 with value: 0.9351345896720886.
[I 2025-11-11 03:53:41,112] Trial 6 finished with value: 0.10532772541046143 and parameters: {'lr': 0.03191925506733783, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 192}. Best is trial 2 with value: 0.9351345896720886.
[I 2025-11-11 03:53:41,151] Trial 7 finished with value: 0.7888405323028564 and parameters: {'lr': 0.006783248358142839, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 98}. Best is trial 2 with value: 0.9351345896720886.


...

Epoch 1, Loss: 107.6654
Epoch 2, Loss: 18.0196
Epoch 3, Loss: 11.4957
Epoch 4, Loss: 4.6966
Epoch 5, Loss: 2.2075
Epoch 6, Loss: 4.3566
Epoch 7, Loss: 2.9115
Epoch 8, Loss: 1.8029
Epoch 9, Loss: 0.8005
Epoch 10, Loss: 0.9919
Epoch 1, Loss: 0.6975
Epoch 2, Loss: 0.6772
Epoch 3, Loss: 0.6695
Epoch 4, Loss: 0.6858
Epoch 5, Loss: 0.6713
Epoch 6, Loss: 0.6788
Epoch 7, Loss: 0.6644
Epoch 8, Loss: 0.6662
Epoch 9, Loss: 0.6598
Epoch 10, Loss: 0.6599
Epoch 1, Loss: 0.6949
Epoch 2, Loss: 0.6865
Epoch 3, Loss: 0.6848
Epoch 4, Loss: 0.6886
Epoch 5, Loss: 0.6909
Epoch 6, Loss: 0.6871
Epoch 7, Loss: 0.6919
Epoch 8, Loss: 0.6898
Epoch 9, Loss: 0.6823
Epoch 10, Loss: 0.6848


[I 2025-11-11 03:54:14,220] Trial 494 finished with value: 0.162944495677948 and parameters: {'lr': 2.5534207467598907e-05, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 53}. Best is trial 37 with value: 0.9988620281219482.
[I 2025-11-11 03:54:14,295] Trial 495 finished with value: 0.5265712738037109 and parameters: {'lr': 0.0061238231640783, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 184}. Best is trial 37 with value: 0.9988620281219482.
[I 2025-11-11 03:54:14,362] Trial 496 finished with value: 0.3313745856285095 and parameters: {'lr': 2.0449717981971525e-05, 'optimizer': 'RMSprop', 'batch_size': 64, 'hidden_size': 139}. Best is trial 37 with value: 0.9988620281219482.


Epoch 1, Loss: 0.6909
Epoch 2, Loss: 0.6816
Epoch 3, Loss: 0.6937
Epoch 4, Loss: 0.6851
Epoch 5, Loss: 0.6830
Epoch 6, Loss: 0.6755
Epoch 7, Loss: 0.6792
Epoch 8, Loss: 0.6829
Epoch 9, Loss: 0.6878
Epoch 10, Loss: 0.6716
Epoch 1, Loss: 1.4682
Epoch 2, Loss: 0.8385
Epoch 3, Loss: 0.7211
Epoch 4, Loss: 0.6347
Epoch 5, Loss: 0.6598
Epoch 6, Loss: 0.6147
Epoch 7, Loss: 0.5958
Epoch 8, Loss: 0.6466
Epoch 9, Loss: 0.6739
Epoch 10, Loss: 0.6274
Epoch 1, Loss: 0.7008
Epoch 2, Loss: 0.6979
Epoch 3, Loss: 0.6997
Epoch 4, Loss: 0.7026
Epoch 5, Loss: 0.6930
Epoch 6, Loss: 0.6983
Epoch 7, Loss: 0.7020
Epoch 8, Loss: 0.6968
Epoch 9, Loss: 0.6928
Epoch 10, Loss: 0.7003


[I 2025-11-11 03:54:14,432] Trial 497 finished with value: 0.336479127407074 and parameters: {'lr': 0.08649534108642627, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 152}. Best is trial 37 with value: 0.9988620281219482.
[I 2025-11-11 03:54:14,497] Trial 498 finished with value: 0.8130258321762085 and parameters: {'lr': 0.0026127497657703544, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 206}. Best is trial 37 with value: 0.9988620281219482.
[I 2025-11-11 03:54:14,548] Trial 499 finished with value: 0.7064523696899414 and parameters: {'lr': 3.066285967589993e-05, 'optimizer': 'RMSprop', 'batch_size': 64, 'hidden_size': 109}. Best is trial 37 with value: 0.9988620281219482.


Epoch 1, Loss: 80.2036
Epoch 2, Loss: 8.8681
Epoch 3, Loss: 2.1445
Epoch 4, Loss: 0.7953
Epoch 5, Loss: 0.6377
Epoch 6, Loss: 0.7324
Epoch 7, Loss: 0.9335
Epoch 8, Loss: 0.7950
Epoch 9, Loss: 0.7063
Epoch 10, Loss: 0.7104
Epoch 1, Loss: 0.8780
Epoch 2, Loss: 0.6768
Epoch 3, Loss: 0.6492
Epoch 4, Loss: 0.6557
Epoch 5, Loss: 0.6408
Epoch 6, Loss: 0.6992
Epoch 7, Loss: 0.6754
Epoch 8, Loss: 0.6692
Epoch 9, Loss: 0.6438
Epoch 10, Loss: 0.6653
Epoch 1, Loss: 0.6799
Epoch 2, Loss: 0.6753
Epoch 3, Loss: 0.6790
Epoch 4, Loss: 0.6704
Epoch 5, Loss: 0.6749
Epoch 6, Loss: 0.6727
Epoch 7, Loss: 0.6731
Epoch 8, Loss: 0.6794
Epoch 9, Loss: 0.6777
Epoch 10, Loss: 0.6760
Best trial FrozenTrial(number=37, state=<TrialState.COMPLETE: 1>, values=[0.9988620281219482], datetime_start=datetime.datetime(2025, 11, 11, 3, 53, 42, 575847), datetime_complete=datetime.datetime(2025, 11, 11, 3, 53, 42, 629533), params={'lr': 2.4917440739386585e-05, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 210}, use

# 아키텍처 구성

In [21]:
def search_hyperparam(trial: optuna.trial.Trial):
    """Optuna를 사용하여 하이퍼파라미터를 탐색"""
    optimizer = trial.suggest_categorical("optimizer", ["Adam", "SGD", "Adagrad"])

    # 옵티마이저에 따른 다른 하이퍼파라미터 설정
    if optimizer == "Adam":
        lr = trial.suggest_float(name="lr", low=1e-5, high=1e-3)
        beta1 = trial.suggest_float(name="beta1", low=0.8, high=0.95)  # Beta1 파라미터
        beta2 = trial.suggest_float(name="beta2", low=0.9, high=0.9999)  # Beta2 파라미터
    elif optimizer == "SGD":
        lr = trial.suggest_float(name="lr", low=1e-6, high=1e-4)
        momentum = trial.suggest_float(name="momentum", low=0.0, high=0.95)  # Momentum 파라미터
    elif optimizer == "Adagrad":
        lr = trial.suggest_float(name="lr", low=1e-5, high=1e-2)
        epsilon = trial.suggest_float(name="epsilon", low=1e-8, high=1e-4)  # Epsilon 파라미터

    return {
        # Optuna에서 주요 타입 정의
        "lr": trial.suggest_float("learning_rate", low=1e-5, high=1e-2, log=True),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "Adagrad"]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        "epochs": trial.suggest_int("epochs", 10, 30, step=10),
        "activation": trial.suggest_categorical(name="m2/activation", choices=["ReLU", "ReLU6", "Hardswish"]),
    }

In [22]:
# Optuna Study 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# 최적의 trial 출력
print(f"Best trial: {study.best_trial}")

[I 2025-11-11 03:54:14,572] A new study created in memory with name: no-name-b1e093bb-46fe-4e6e-8bfd-f07603d750f4


Epoch 1, Loss: 0.6981
Epoch 2, Loss: 0.7221
Epoch 3, Loss: 0.6531


[I 2025-11-11 03:54:14,629] Trial 0 finished with value: 0.7179260849952698 and parameters: {'optimizer': 'Adagrad', 'lr': 0.0035687056261235195, 'epsilon': 2.6140920736203454e-05, 'learning_rate': 0.001751054169458058, 'batch_size': 32, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 86}. Best is trial 0 with value: 0.7179260849952698.
[I 2025-11-11 03:54:14,693] Trial 1 finished with value: 0.5991528034210205 and parameters: {'optimizer': 'Adam', 'lr': 0.0001470522228010742, 'beta1': 0.8794351230957622, 'beta2': 0.9494430592054433, 'learning_rate': 0.00657698408004978, 'batch_size': 32, 'epochs': 10, 'm2/activation': 'Hardswish', 'hidden_size': 91}. Best is trial 0 with value: 0.7179260849952698.
[I 2025-11-11 03:54:14,765] Trial 2 finished with value: 0.3074514865875244 and parameters: {'optimizer': 'Adagrad', 'lr': 0.004363711714619402, 'epsilon': 2.8182322112649964e-05, 'learning_rate': 1.89177902783519e-05, 'batch_size': 16, 'epochs': 20, 'm2/activation': 'ReLU6', 'hidden_s

Epoch 4, Loss: 0.6774
Epoch 5, Loss: 0.6825
Epoch 6, Loss: 0.6647
Epoch 7, Loss: 0.6816
Epoch 8, Loss: 0.6833
Epoch 9, Loss: 0.7197
Epoch 10, Loss: 0.6721
Epoch 1, Loss: 0.6847
Epoch 2, Loss: 0.6604
Epoch 3, Loss: 0.6856
Epoch 4, Loss: 0.7030
Epoch 5, Loss: 0.6737
Epoch 6, Loss: 0.6856
Epoch 7, Loss: 0.6797
Epoch 8, Loss: 0.6248
Epoch 9, Loss: 0.6398
Epoch 10, Loss: 0.6385
Epoch 1, Loss: 0.6877
Epoch 2, Loss: 0.6864
Epoch 3, Loss: 0.6853
Epoch 4, Loss: 0.6867
Epoch 5, Loss: 0.6895
Epoch 6, Loss: 0.6846
Epoch 7, Loss: 0.6902
Epoch 8, Loss: 0.6896
Epoch 9, Loss: 0.6903
Epoch 10, Loss: 0.6909
Epoch 1, Loss: 0.6840
Epoch 2, Loss: 0.6908
Epoch 3, Loss: 0.6841
Epoch 4, Loss: 0.6796
Epoch 5, Loss: 0.6613
Epoch 6, Loss: 0.6631
Epoch 7, Loss: 0.6512


[I 2025-11-11 03:54:14,817] Trial 3 finished with value: 0.38050591945648193 and parameters: {'optimizer': 'Adagrad', 'lr': 0.0019178951830554254, 'epsilon': 6.206337955646836e-05, 'learning_rate': 0.003910612966309187, 'batch_size': 32, 'epochs': 20, 'm2/activation': 'Hardswish', 'hidden_size': 249}. Best is trial 0 with value: 0.7179260849952698.
[I 2025-11-11 03:54:14,874] Trial 4 finished with value: 0.03533315658569336 and parameters: {'optimizer': 'SGD', 'lr': 7.819118168416212e-05, 'momentum': 0.6375797856669386, 'learning_rate': 1.5158686891679567e-05, 'batch_size': 16, 'epochs': 30, 'm2/activation': 'ReLU6', 'hidden_size': 87}. Best is trial 0 with value: 0.7179260849952698.
[I 2025-11-11 03:54:14,901] Trial 5 finished with value: 0.6122182607650757 and parameters: {'optimizer': 'SGD', 'lr': 6.737137225202176e-05, 'momentum': 0.5619352989656838, 'learning_rate': 1.2253709333900417e-05, 'batch_size': 64, 'epochs': 20, 'm2/activation': 'Hardswish', 'hidden_size': 215}. Best is t

Epoch 8, Loss: 0.6418
Epoch 9, Loss: 0.6291
Epoch 10, Loss: 0.6421
Epoch 1, Loss: 0.6927
Epoch 2, Loss: 0.6971
Epoch 3, Loss: 0.6995
Epoch 4, Loss: 0.6952
Epoch 5, Loss: 0.6962
Epoch 6, Loss: 0.7032
Epoch 7, Loss: 0.7015
Epoch 8, Loss: 0.7029
Epoch 9, Loss: 0.6839
Epoch 10, Loss: 0.6940
Epoch 1, Loss: 0.7203
Epoch 2, Loss: 0.7272
Epoch 3, Loss: 0.7331
Epoch 4, Loss: 0.7197
Epoch 5, Loss: 0.7264
Epoch 6, Loss: 0.7271
Epoch 7, Loss: 0.7279
Epoch 8, Loss: 0.7249
Epoch 9, Loss: 0.7252
Epoch 10, Loss: 0.7261
Epoch 1, Loss: 0.7039
Epoch 2, Loss: 0.6904
Epoch 3, Loss: 0.6891
Epoch 4, Loss: 0.6695
Epoch 5, Loss: 0.6619
Epoch 6, Loss: 0.6600
Epoch 7, Loss: 0.6426
Epoch 8, Loss: 0.6441
Epoch 9, Loss: 0.6219
Epoch 10, Loss: 0.6288
Epoch 1, Loss: 0.7074
Epoch 2, Loss: 0.7084
Epoch 3, Loss: 0.7193
Epoch 4, Loss: 0.7159
Epoch 5, Loss: 0.7102
Epoch 6, Loss: 0.7055
Epoch 7, Loss: 0.6983
Epoch 8, Loss: 0.6966
Epoch 9, Loss: 0.6973
Epoch 10, Loss: 0.7124
Epoch 1, Loss: 0.7102
Epoch 2, Loss: 0.7098
Epoch

[I 2025-11-11 03:54:15,024] Trial 8 finished with value: 0.5672802925109863 and parameters: {'optimizer': 'Adam', 'lr': 0.00034628746762156097, 'beta1': 0.8244457180525814, 'beta2': 0.9054674353946285, 'learning_rate': 0.00028938042191229736, 'batch_size': 32, 'epochs': 10, 'm2/activation': 'ReLU', 'hidden_size': 92}. Best is trial 0 with value: 0.7179260849952698.
[I 2025-11-11 03:54:15,065] Trial 9 finished with value: 0.48207026720046997 and parameters: {'optimizer': 'SGD', 'lr': 3.513200727237854e-05, 'momentum': 0.8367432160304066, 'learning_rate': 7.866927949469143e-05, 'batch_size': 32, 'epochs': 30, 'm2/activation': 'Hardswish', 'hidden_size': 39}. Best is trial 0 with value: 0.7179260849952698.


Epoch 8, Loss: 0.6942
Epoch 9, Loss: 0.6930
Epoch 10, Loss: 0.6878
Epoch 1, Loss: 0.7055
Epoch 2, Loss: 0.6957
Epoch 3, Loss: 0.6981
Epoch 4, Loss: 0.7078
Epoch 5, Loss: 0.6925
Epoch 6, Loss: 0.7175
Epoch 7, Loss: 0.6917
Epoch 8, Loss: 0.6871
Epoch 9, Loss: 0.6930
Epoch 10, Loss: 0.6960
Best trial: FrozenTrial(number=0, state=<TrialState.COMPLETE: 1>, values=[0.7179260849952698], datetime_start=datetime.datetime(2025, 11, 11, 3, 54, 14, 576883), datetime_complete=datetime.datetime(2025, 11, 11, 3, 54, 14, 628942), params={'optimizer': 'Adagrad', 'lr': 0.0035687056261235195, 'epsilon': 2.6140920736203454e-05, 'learning_rate': 0.001751054169458058, 'batch_size': 32, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 86}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'optimizer': CategoricalDistribution(choices=('Adam', 'SGD', 'Adagrad')), 'lr': FloatDistribution(high=0.01, log=False, low=1e-05, step=None), 'epsilon': FloatDistribution(high=0.0001, log=False, l

In [23]:
# Optuna의 큰 장점은 조건부 파라미터(Conditional Parameter)를 설정 할수 있는 것
# 분기 처리를 통해 Conditional 을 구성할 수 있음
# 어떤 옵티마이저를 샘플링 하냐에 따라서 다른 탐색 공간을 가져감

# 고정된 n,nc,rc

In [24]:
def objective(trial: optuna.trial.Trial) -> float:
    """Optuna 최적화 목표 함수"""
    hyperparams = search_hyperparam(trial)
    model = search_model(trial)
    train_loader = DataLoader([(torch.rand(10), torch.randint(0, 2, (1,))) for _ in range(100)], batch_size=hyperparams["batch_size"], shuffle=True)
    trained_model = train_model(model, hyperparams, train_loader)
    score = evaluate(trained_model)
    return score

In [25]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [26]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-11-11 03:54:15,104] A new study created in memory with name: no-name-a4380182-8b16-4432-ba58-1ddff4c47d96
[I 2025-11-11 03:54:15,108] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'InvertedResidualv2', 'reduction_cell': 'AvgPool', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,110] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'AvgPool', 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,111] Trial 2 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'AvgPool', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,113] Trial 3 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'InvertedResidualv3', 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,115] Trial 4 finished with value: 8.0 and parameters: {'normal_cell': 'Conv', 'normal_cell/out_channels': 45

# 고정된 nc,rc N은 점점 커지게

In [27]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    n1, n2, n3 = n, 2*n, 3*n
    model.append([n1, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n2, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n3, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [28]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-11-11 03:54:15,152] A new study created in memory with name: no-name-d3c6e0e7-c156-4e0b-ba68-3632d1429321
[I 2025-11-11 03:54:15,155] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'DWConv', 'reduction_cell': 'MaxPool', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,158] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'AvgPool', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,159] Trial 2 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'InvertedResidualv3', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,162] Trial 3 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'MaxPool', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,164] Trial 4 finished with value: 8.0 and parameters: {'normal_cell': 'InvertedResidualv2', 'reduction_cell': 'InvertedR

In [29]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    n1 = trial.suggest_int("n_repeat", 1, 3)
    n2 = trial.suggest_int("n_repeat", 2, 5)
    n3 = trial.suggest_int("n_repeat", 3, 8)

    model.append([n1, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n2, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n3, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [30]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-11-11 03:54:15,198] A new study created in memory with name: no-name-e201c4d9-c2ff-4f6a-b909-c798e062e7d4
/usr/local/lib/python3.12/dist-packages/optuna/trial/_trial.py:686: RuntimeWarning: Inconsistent parameter values for distribution with name "n_repeat"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 1, 'high': 3}
  warnings.warn(
[I 2025-11-11 03:54:15,204] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'InvertedResidualv2', 'reduction_cell': 'AvgPool', 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,207] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'MaxPool', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-11-11 03:54:15,210

# ModuleGenerator의
# ymal에서 Model 만들기

In [31]:
class ModelParser:
	def __init__(self, cfg: str = ".model.yaml") -> None:
		with open(cfg) as f:
			self.cfg = yaml.load(f, Loader=yaml.FullLoader)

		self.in_channel = self.cfg["input_channel"],

		self.depth_multiply = self.cfg["depth_multiple"]
		self.width_multiply = self.cfg["width_multiple"]

		self.model_cfg = self.cfg["backbone"]
		self.model = self._parse_model()

In [34]:
def _parse_model(self) -> nn.Module:
    """Parse model."""
    layers: List[nn.Module] = []
    in_channel = self.in_channel
    for i, (repeat, module, args) in enumerate(self.model_cfg):
        repeat = max(round(repeat * self.depth_multiply), 1) if repeat > 1 else repeat
        module_generator = ModuleGenerator(module)(
            *args,
            width_multiply=self.width_multiply
        )
        m = module_generator(repeat=repeat)

        layers.append(m)
        in_channel = module_generator.out_channel

    parsed_model = nn.Sequential(*layers)
    return parsed_model

In [35]:
# depth_multiple 과 width_multiple 이라는 파라미터는 MobileNetV1 에서 처음 등장한 컨셉 간략하게 설명드리자면
# 하나의 모델을 여러개의 Device 에서 쓰기 위해 유연함을 주는 개념
# depth_multiple 은 반복횟수를 조절
# width_multiple 은 채널의 크기를 조절
# 두 파라미터 depth_multiple 과 width_multiple 은 모델의 크기를 조율하는데 효과적인 파라미터

In [39]:
class Bottleneck(nn.Module):
  """Simplified bottleneck block."""
  def __init__(
			self,
			in_channel: int,
			out_channels: int,
			shortcut=True,
			groups: int = 1,
			expansion: float = 0.5,
			activation: Union[str, None] = "ReLU"
		) -> None:
    """Initialize."""
    super().__init__()
    expansion_channel = int(out_channels * expansion)

    self.conv1 = Conv(in_channel, expansion_channel, 1, 1, activation=activation)
    self.conv2 = Conv(expansion_channel, out_channels, 3, 1, groups=groups)
    self.shortcut = shortcut and in_channel == out_channels

  def forward(self, x: torch. Tensor) -> torch. Tensor:
    """Forward."""
    out = self.conv2( self.conv1(x))
    if self.shortcut:
    	out = out + x
    return out

In [ ]:
class BottleneckGenerator(GeneratorAbstract):
	"""Bottleneck block generator."""
	def __init__(self, *args, **kwargs):
		super().__init__(*args, **kwargs)

	@property
	def out_channel(self) -> int:
		"""Get out channel size."""
		return self._get_divisible_channel(self.args[0] * self.width_multiply)

	@property
	def base_module(self) -> nn.Module:
		"""Returns module class from src.modules based on the class name."""
		return getattr(__import__("src.modules", fromlist=[""]), self.name)

	def __call__(self, repeat: int = 1):
		repeat_args = [self.in_channel, self.in_channel, *self.args[1:]]
		args = [self.in_channel, self.out_channel, *self.args[1:]]
		module = []
		if repeat > 1:
			for _ in range(repeat-1):
				module.append(self.base_module(*repeat_args))
		module.append(self.base_module(*args))
		return self._get_module(module)

In [ ]:
# 아웃풋 채널(self.args[0])에 width_multiply를 곱한 값이 out_channel의 값
# call 함수에서는 repeat 값을 입력받아서 주어진 반복 횟수만큼 모듈을 리스트에 추가하는 것을 확인